# Analyze scraped NRSR values

To get all datasets used in this notebook run:

```bash
# get voting data
python src/main.py --type voting \
                   --start-id 51426 \                                  
                   --end-id 55902 \                                   
                   --log-file data/raw/voting_data_51426-55902.log \
                   --save-to data/raw/voting_data_51426-55902.json

# get member bios
python src/main.py --type member \
                   --input-file data/raw/voting_data_51426-55902.json \
                   --log-file data/raw/members_data_51426-55902.log \ 
                   --save-to data/raw/members_data_51426-55902.json 

# get member election results
python src/main.py --type election \
                    --input-file https://volby.statistics.sk/nrsr/nrsr2023/files/xlsx/NRSR2023_SK_tab07a.xlsx \
                    --save-to data/interim/member_elections.xlsx

# get document info related to voting
python src/main.py --type document \
                   --input-file data/raw/voting_data_51426-55902.json \
                   --log-file data/raw/document_data_51426-55902.log \
                   --save-to data/raw/document_data_51426-55902.xlsx

# convert the data to input file use here
python src/convert/convert_to_excel.py --input-voting data/raw/voting_data_51426-55902.json \
                                       --input-member data/raw/members_data_51426-55902.json \
                                       --input-election data/interim/member_elections.xlsx \
                                       --input-document data/raw/document_data_51426-55902.xlsx \
                                       --output-file data/interim/voting_member_election_document_51426-55902.xlsx
```

## Import the required libraries

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime

## Processing settings

In [2]:
input_file = '../../data/interim/voting_member_election_document_51426-55902.xlsx'
min_date = datetime(2024, 1, 1)
max_date = datetime(2024, 12, 31)
coalition_votes_nr = 79

## Read data

In [ ]:
df = pd.read_excel(input_file)
df['cas_hlasovania'] = pd.to_datetime(df['cas_hlasovania'], dayfirst=True)
df = df[(df['cas_hlasovania'] >= min_date) & (df['cas_hlasovania'] <= max_date)]
df.head()

## Basic statistics

In [ ]:
columns_voting = [
  'voting_id', 'cas_hlasovania', 'schodza', 'cislo_schodze', 'cislo_hlasovania', 'nazov_hlasovania', 'vysledok_hlasovania', 'schvalene', 'pritomni', 'hlasujucich', 'za_hlasovalo', 'proti_hlasovalo', 'zdrzalo_sa', 'nehlasovalo', 'nepritomni'
]
dfa = df[columns_voting].groupby('voting_id').max()
dfa.head()

In [ ]:
basic_stats = {
    'totalVotes': dfa.shape[0],
    'totalVotesCount': dfa.hlasujucich.sum(),
    'votesPassed': int(dfa['schvalene'].str.count('ano').sum()),
    'votesFailed': int(dfa['schvalene'].str.count('nie').sum()),
    'votesInvalid': int(df['hlas'].str.count('neplatny').sum()),
    'votesOverThreshold': dfa[dfa['za_hlasovalo'] >= coalition_votes_nr].shape[0],
}
basic_stats

## Votings in graphs

### Monthly votes

In [ ]:
# Add a new column for the month
dfa['month'] = dfa['cas_hlasovania'].dt.to_period('M')

# Group by month and schvalene, then count the number of votes
monthly_votes = dfa.groupby(['month', 'schvalene']).size().unstack(fill_value=0)

monthly_votes
# Convert the month to a string format
monthly_votes.index = monthly_votes.index.strftime('%b')

# Convert to list of dictionaries
monthly_votes_list = monthly_votes.reset_index().rename(columns={'month': 'date'}).to_dict(orient='records')
monthly_votes_list

### Vote count distribution

In [ ]:
temp = dfa[['za_hlasovalo']].copy()
temp['za_hlasovalo'] = temp['za_hlasovalo'].clip(lower=0, upper=150)

votes_distribution = pd.DataFrame({'votes': [i  for i in range(0,151)]})
votes_distribution = votes_distribution.assign(count = 0)
for vote in votes_distribution['votes']:
    votes_distribution.loc[vote, 'count'] = temp[temp['za_hlasovalo'] == vote].shape[0]

votes_distribution_list = votes_distribution.to_dict(orient='records')
votes_distribution_list

### Vote by hour distribution

In [ ]:
dfa['hour'] = dfa['cas_hlasovania'].dt.hour

# Group by day and hour and count the number of votings
votings_per_day_hour = dfa.groupby(['hour']).size().reset_index(name='count')

# envorce hour from 0 to 23 - use merge to do that
votings_per_day_hour = pd.merge(pd.DataFrame({'hour': range(24)}), votings_per_day_hour, on='hour', how='left').fillna(0)
votings_per_day_hour['count'] = votings_per_day_hour['count'].astype(int)

# Convert the result to a list of dictionaries
votings_per_day_hour_list = votings_per_day_hour.to_dict(orient='records')
votings_per_day_hour_list

## Members - fun facts

In [ ]:
member_columns = [
    'poslanec_id', 'poslanec_meno', 'poslanec_priezvisko', 'poslanec_priezvisko_meno', 'poslanec_titul', 'kandidoval_za', 'poslanec_narodeny', 'poslanec_narodnost', 'poslanec_bydlisko', 'poslanec_kraj', 'poslanec_email', 'poslanec_www', 'poslanec_photo', 'poslanec_clenstvo', 'poslanec_titul_pocet', 'poslanec_volby_hlasov', 'poslanec_volby_hlasov_podiel', 'poslanec_volby_poradie', 'poslanec_volby_poradie_listok'
]
dfm = df.drop_duplicates(subset='poslanec_id')[member_columns]
dfm['poslanec_kraj'] = dfm['poslanec_kraj'].str.replace(' kraj', '')

dfm['narodeny_datetime'] = pd.to_datetime(dfm['poslanec_narodeny'], format='%d. %m. %Y')

current_date = datetime.now()
dfm['poslanec_vek'] = dfm['narodeny_datetime'].apply(lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day)))

dfm.head()

### Birthday votes

In [ ]:
df['poslanec_narodeny_datetime'] = pd.to_datetime(df['poslanec_narodeny'], dayfirst=True)

df['vote_day'] = df['cas_hlasovania'].dt.day
df['vote_month'] = df['cas_hlasovania'].dt.month
df['birth_day'] = df['poslanec_narodeny_datetime'].dt.day
df['birth_month'] = df['poslanec_narodeny_datetime'].dt.month

# Compare the day and month of the voting date with the birthday
birthday_votes = df[(df['vote_day'] == df['birth_day']) & (df['vote_month'] == df['birth_month'])]

# Count the number of matches
birthday_vote_count = birthday_votes.shape[0]

birthday_votes_dict = {
    'totalMembers': df['poslanec_id'].nunique(),
    'uniqueMembers': birthday_votes['poslanec_id'].nunique(),
    'birthdayVotes': birthday_vote_count,
    'birthdayVotesPercentage': np.round((birthday_vote_count / df.shape[0])*100,3),
}
birthday_votes_dict


### Most restrained member

In [ ]:
zdrzal_votes = df[df['hlas'] == 'zdrzal']

# Group by the member identifier and count the occurrences of "zdrzal"
zdrzal_counts = zdrzal_votes.groupby('poslanec_id').size().reset_index(name='count')

# Identify the member with the highest count
most_zdrzal_member = zdrzal_counts.loc[zdrzal_counts['count'].idxmax()]

most_zdrzal_member_dict = {
    'memberID': most_zdrzal_member['poslanec_id'],
    'memberVotes': most_zdrzal_member['count'],
    'votesPercentage': np.round((most_zdrzal_member['count'] / dfa.shape[0])*100,3),
    'memberName': df[df['poslanec_id'] == most_zdrzal_member['poslanec_id']]['poslanec_priezvisko_meno'].iloc[-1],
    'memberParty': df[df['poslanec_id'] == most_zdrzal_member['poslanec_id']]['kandidoval_za'].iloc[-1],
}
most_zdrzal_member_dict

In [ ]:
dfm

In [13]:
# compare_file = '../../data/processed/data_hlasovania_2024_12_23_05_04.csv'
# x = pd.read_csv(compare_file, sep='\t')

### Member with minimum number of votes (in election)

In [ ]:
# member with lelast number in the poslanec_volby_hlasov column
temp = dfm.sort_values(by='poslanec_volby_hlasov', ascending=True).head(1)

least_votes_member_dict = {
    'memberID': temp['poslanec_id'].iloc[0],
    'memberVotes': temp['poslanec_volby_hlasov'].iloc[0],
    'votesPercentage': temp['poslanec_volby_hlasov_podiel'].iloc[0],
    'memberName': temp['poslanec_priezvisko_meno'].iloc[0],
    'memberParty': temp['kandidoval_za'].iloc[0],
}
least_votes_member_dict



### Members by 'kraj'

In [ ]:
member_kraj = dfm[['poslanec_kraj','poslanec_id']].groupby('poslanec_kraj').count().reset_index().rename(columns={'poslanec_kraj':'kraj','poslanec_id':'count'})
member_kraj_list = member_kraj.sort_values(by='count',ascending=False).to_dict(orient='records')
member_kraj_list

### Members by academic titles

In [ ]:
member_title = dfm[['poslanec_id','poslanec_titul_pocet']].groupby('poslanec_titul_pocet').count().reset_index().rename(columns={'poslanec_id':'count','poslanec_titul_pocet':'titlesNumber'})
member_title_list = member_title.sort_values(by='count',ascending=False).to_dict(orient='records')
member_title_list

### Members by age

In [ ]:
member_age = dfm[['poslanec_id','poslanec_vek']].groupby('poslanec_vek').count().reset_index().rename(columns={'poslanec_id':'count','poslanec_vek':'vek'})
member_age_list = member_age.sort_values(by='vek', ascending=True).to_dict(orient='records')
member_age_list